In [ ]:
!pip install -U -I git+https://github.com/raghakot/keras-vis.git
!pip install scipy==1.1.0
!pip install nilearn

In [1]:
import tensorflow as tf
import keras
from keras.models import load_model
from keras import models
from keras.models import Model 
import numpy as np
import nibabel as nib
from keras import backend as K

import os
import matplotlib.cm as cm
from vis.visualization import visualize_cam
from vis.visualization import visualize_saliency, overlay
from vis.utils import utils
from keras import activations
from nilearn import plotting

from nilearn.datasets import load_mni152_template
from nilearn.image import resample_to_img,load_img


from scipy.stats import zscore
from nilearn.image import threshold_img
from scipy.io import savemat

from nilearn.image import load_img, index_img
from nilearn.image import resample_to_img
from nilearn.datasets import load_mni152_template

ImportError: cannot import name '_Conv'

In [2]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [3]:
import keras
keras.__version__

'2.4.3'

In [5]:
def get_gradCAM_map(input_data,model,layer_name,penultimate_layer_name,trial,filter_indices=None,
                    backprop_modifier=None,output_path=None,resample=True,threshold=True,
                    fname='activation-3',plot=True,affine_mat=None,hdr=None):
    
    layer_idx = utils.find_layer_idx(model, layer_name)
    penultimate_layer = utils.find_layer_idx(model, penultimate_layer_name)
    if isinstance(input_data,str):
        data = np.array(nib.load(input_data).get_fdata())
        affine_mat = nib.load(input_data).affine
        hdr = nib.load(input_data).header
    else:
        data = input_data
#     if data.shape
#     img = np.rollaxis(data, 3, 0)[...,None]
    img = np.rollaxis(data, 0, 0)[...,None]
    grads = visualize_cam(model, layer_idx, filter_indices, seed_input=img, 
                          penultimate_layer_idx=penultimate_layer, backprop_modifier=None)   
#     grads = visualize_cam(model, layer_idx, filter_indices, seed_input=img[10:15,:,:,:,:], 
#                           penultimate_layer_idx=penultimate_layer, backprop_modifier=None)
    template = load_mni152_template()
    if resample:
        grads_ = nib.Nifti1Image(grads, affine_mat, hdr)
        resampled_grads_img = resample_to_img(grads_, template)
        resampled_fname = os.path.join(output_path,'trial-'+str(trial)+'-'+fname+'.nii')
        resampled_grads_img.to_filename(resampled_fname)
        if plot:
            resampled_plot_fname = os.path.join(output_path,'trial-'+str(trial)+'-'+fname)
            plotting.plot_stat_map(resampled_grads_img, display_mode='z', cut_coords=[36, -27, 60],
                           title='Unthresholded beta-map', 
                           output_file=resampled_plot_fname,colorbar=True)
        
    if threshold:
        threshold_percentile_img = threshold_img(resampled_fname, threshold='97%', copy=False)
        thresholded_fname = os.path.join(output_path,'trial-'+str(trial)+'-'+fname+'-threshold.nii')
        threshold_percentile_img.to_filename(thresholded_fname)
        print("Thresholded image saved!")
        if plot:
            threshold_plot_fname = os.path.join(output_path,'trial-'+str(trial)+'-'+fname+'-threshold')
            plotting.plot_stat_map(threshold_percentile_img, display_mode='z', cut_coords=[36, -27, 60],
                                   title='Thresholded beta-map with 97% percentile', 
                                   output_file=threshold_plot_fname,colorbar=True)
       
    return grads,resampled_grads_img

In [ ]:
data = '/home/tanmay/JHU/project/data/BMRK4/bmrk4_st_sub011-2.nii'

layers = ["activation_3","activation_5","activation_7","activation_9","activation_11","activation_12"]
penultimate_layer_name = 'conv3d_1'
result_path = '/home/tanmay/JHU/project/deep-mediation/results/nov2020/visualize_model'

num_trials = nib.load(data).get_fdata().shape[-1]
header = nib.load(data).header
affine_matrix = nib.load(data).affine

for trial in range(num_trials):
    input_data = np.array(nib.load(data).get_fdata())[:,:,:,trial]
    print("Processing %s trial..."%trial)
    for layer_name in layers:
        print("Processing %s"%layer_name)
        grads,resampled_grads_img = get_gradCAM_map(input_data,model,layer_name,penultimate_layer_name,trial,
                                                    filter_indices=None,backprop_modifier=None,
                                                    output_path=result_path,resample=True,
                                                    threshold=True,fname=layer_name,plot=True,
                                                    affine_mat=affine_matrix,hdr=header)
